## Hb 감소 1이상인 시점으로부터 24시간 데이터 추출



## SQL 쿼리 시작 (df_event / df_non_event)

In [2]:
from google.colab import auth, drive

auth.authenticate_user()
print('Authenticated')

drive.mount('/content/drive')
print('Mounted')

Authenticated
Mounted at /content/drive
Mounted


In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pickle

In [3]:
project_id = 'mimic3-330515'

In [59]:
%load_ext google.colab.data_table

In [60]:
# Hb 감소 환자들
# charttime과 storetime??? 어떤걸로 쓸지  -> chrattime
# time_event는 발생 전 시간이 24시간 내에 발생한것만 추린 것임, 1일때가 24시간 이내.
# icd code의 경우 리스트 정해주면 따로 추출조건 만들 수 있음 

%%bigquery --project mimic3-330515 df
WITH hb_table AS(
  SELECT icu.hadm_id, charttime, valuenum, flag,
        LAG(valuenum) OVER(PARTITION BY icu.hadm_id ORDER BY charttime) AS prev_valuenum,
        LAG(charttime) OVER(PARTITION BY icu.hadm_id ORDER BY charttime) AS prev_valuetime,
        DATE_DIFF(DATE(icu.intime), DATE(pat.dob), YEAR) AS age,
        TIMESTAMP_DIFF(icu.outtime, icu.intime, DAY) AS icu_length_of_stay,
        icu.intime,
        icu.outtime,
        CASE WHEN (DATETIME_DIFF(charttime, LAG(charttime) OVER(PARTITION BY icu.hadm_id ORDER BY charttime), HOUR) < 24
            AND DATETIME_DIFF(charttime, LAG(charttime) OVER(PARTITION BY icu.hadm_id ORDER BY charttime), HOUR) > 12 ) THEN LAG(valuenum) OVER(PARTITION BY icu.hadm_id ORDER BY charttime)
            ELSE NULL END AS prev_value_in24h, ##
        CASE WHEN ((LAG(valuenum) OVER(PARTITION BY icu.hadm_id ORDER BY charttime)) - valuenum >= 1) THEN 1 ##
            ELSE 0 END AS event,
        CASE WHEN (DATETIME_DIFF(charttime, LAG(charttime) OVER(PARTITION BY icu.hadm_id ORDER BY charttime), HOUR) < 24
            AND DATETIME_DIFF(charttime, LAG(charttime) OVER(PARTITION BY icu.hadm_id ORDER BY charttime), HOUR) > 12) THEN 1
            ELSE 0 END AS time_event
  FROM physionet-data.mimiciii_clinical.labevents as le
  INNER JOIN physionet-data.mimiciii_clinical.icustays AS icu
      ON icu.hadm_id = le.hadm_id
  INNER JOIN `physionet-data.mimiciii_clinical.patients` AS pat
      ON icu.subject_id = pat.subject_id
  INNER JOIN `physionet-data.mimiciii_clinical.diagnoses_icd` AS diag
      ON icu.hadm_id = diag.hadm_id
  WHERE le.ITEMID = 51222
  AND (diag.ICD9_CODE	= '5780' OR diag.ICD9_CODE	= '5781' OR diag.ICD9_CODE	= '5789')
)
SELECT 
  hadm_id, charttime, valuenum, flag, prev_valuenum, prev_valuetime, age, icu_length_of_stay,
  intime, outtime, prev_value_in24h, event, time_event
FROM hb_table
    WHERE icu_length_of_stay > 2
    AND age < 89
    AND age > 18
    AND valuenum > 3
    AND valuenum < 40
    AND intime < charttime
    AND charttime < outtime
ORDER BY hadm_id, charttime

In [61]:
df_event = df.loc[(df['time_event']==1) & (df['event']==1) & (pd.notnull(df['prev_value_in24h'])),:]
df_event = df_event.drop_duplicates(['hadm_id'], keep='first') ## 환자 1명당 1 case 추출
df_event = df_event.assign(charttime_ind=df_event.charttime.dt.round('H'))
df_event = df_event.reset_index(drop=True, col_fill='index')
# df_event = df_event.iloc[0:100,:]

df_non_event = df.loc[(df['time_event']==1) & (df['event']==0),:]
df_non_event = df_non_event.drop_duplicates(['hadm_id'], keep='first')
df_non_event = df_non_event.assign(charttime_ind=df_non_event.charttime.dt.round('H'))
df_non_event = df_non_event.reset_index(drop=True, col_fill='index')
# df_non_event = df_non_event.iloc[0:100,:]

df_total = pd.concat([df_event, df_non_event])

In [62]:
print(len(df_event['hadm_id']))
print(len(df_non_event['hadm_id']))
print(len(df_total['hadm_id']))

425
953
1378


In [ ]:
df_total

,hadm_id,charttime,valuenum,flag,prev_valuenum,prev_valuetime,age,icu_length_of_stay,intime,outtime,prev_value_in24h,event,time_event,charttime_ind
0,100132,2179-03-06 04:54:00,12.6,None,15.0,2179-03-05 14:50:00,74,19,2179-03-05 20:14:43,2179-03-24 11:37:33,15.0,1,1,2179-03-06 05:00:00
1,100199,2185-11-13 00:08:00,11.4,abnormal,15.6,2185-11-12 10:00:00,75,5,2185-11-12 20:40:11,2185-11-17 18:32:50,15.6,1,1,2185-11-13 00:00:00
2,100853,2166-03-28 02:19:00,10.1,abnormal,11.2,2166-03-27 12:08:00,55,4,2166-03-25 19:43:06,2166-03-29 16:49:50,11.2,1,1,2166-03-28 02:00:00
3,101011,2162-05-21 01:45:00,9.7,abnormal,11.2,2162-05-20 02:54:00,68,10,2162-05-16 12:53:28,2162-05-26 21:20:59,11.2,1,1,2162-05-21 02:00:00
4,101170,2106-02-15 01:54:00,9.3,abnormal,11.2,2106-02-14 03:05:00,82,16,2106-02-10 16:02:50,2106-02-26 15:18:32,11.2,1,1,2106-02-15 02:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1139,199365,2111-01-10 18:49:00,9.1,abnormal,10.0,2111-01-10 07:15:00,75,24,2111-01-10 13:40:00,2111-02-03 16:26:00,NaN,0,0,2111-01-10 19:00:00
1140,199544,2108-08-02 15:59:00,6.6,abnormal,5.8,2108-08-02 11:37:00,88,6,2108-08-02 14:52:00,2108-08-08 15:21:00,NaN,0,0,2108-08-02 16:00:00
1141,199589,2146-09-28 15:00:00,10.1,abnormal,NaN,NaT,60,10,2146-09-28 14:45:49,2146-10-08 02:16:51,NaN,0,0,2146-09-28 15:00:00
1142,199780,2144-07-06 02:10:00,12.8,delta,9.5,2144-07-06 01:30:00,52,3,2144-07-03 20:32:58,2144-07-06 22:56:09,NaN,0,0,2144-07-06 02:00:00


In [63]:
def before_event_range(data):
  ptest = data
  pdict = dict()
  for i in ptest['hadm_id'].tolist():
    ptest_slice = ptest[ptest['hadm_id'] == i]
    timerange = []
    for time in ptest_slice['charttime'].tolist():
      timerangesmall = []
      timerangesmall.append(str(time - timedelta(hours=24)))
      timerangesmall.append(str(time))
      timerange.append(timerangesmall)
    pdict[i] = timerange
  return pdict

In [64]:
new_event = before_event_range(df_non_event)

In [65]:
## chart

chart_df = pd.DataFrame()
for id,times in new_event.items():
  for time in times:
    SQL = """SELECT hadm_id,charttime, itemid, valuenum
            FROM physionet-data.mimiciii_clinical.chartevents as ce 
            WHERE ce.HADM_ID = {x}
            AND (ce.charttime between {a} AND {b} )
            AND (ce.ITEMID = 220050 OR ce.ITEMID = 220179 OR ce.ITEMID = 225309 OR ce.ITEMID = 51 OR ce.ITEMID = 455 /*SBP*/
                OR ce.ITEMID = 220051 OR ce.ITEMID = 220180 OR ce.ITEMID = 225310 OR ce.ITEMID = 8368 OR ce.ITEMID = 8441 /*DBP*/
                OR ce.ITEMID = 220210 OR ce.ITEMID = 618 /*Respiratory*/
                OR ce.ITEMID = 220045 OR ce.ITEMID = 211 /*Heart rate*/
                OR ce.ITEMID = 223762 OR ce.ITEMID = 676 OR ce.ITEMID = 677 OR ce.ITEMID = 223761 OR ce.ITEMID = 678 OR ce.ITEMID = 679 /*Temperature*/
                OR ce.ITEMID = 646 /*SpO2*/
                )
            ;""".format(x=id,a= "'{}'".format(time[0]), b="'{}'".format(time[1]))
    adddf = pd.io.gbq.read_gbq(SQL, project_id= project_id, dialect= 'standard')
    chart_df=chart_df.append(adddf)

chart_df = chart_df.assign(charttime_ind=chart_df.charttime.dt.round('H'))


## Urine item_id
##43171, 43173, 43355, 43365, 43373, 43374, 43379, 43380, 43431, 43522, 45304, 43576, 43589, 43811, 43812, 43333, 43638, 43654

In [67]:
chart_df.to_csv("/content/drive/MyDrive/의료인공지능/mimic_test/chartdf_non_event.csv")
df_non_event.to_csv("/content/drive/MyDrive/의료인공지능/mimic_test/df_non_event.csv")

In [ ]:
chart_df.head(50)

,hadm_id,charttime,itemid,valuenum,charttime_ind
0,100132,2179-03-06 04:10:00,8441,43.000000,2179-03-06 04:00:00
1,100132,2179-03-06 02:00:00,455,100.000000,2179-03-06 02:00:00
2,100132,2179-03-05 23:00:00,646,100.000000,2179-03-05 23:00:00
3,100132,2179-03-06 04:20:00,455,58.000000,2179-03-06 04:00:00
4,100132,2179-03-06 02:25:00,8441,45.000000,2179-03-06 02:00:00
5,100132,2179-03-06 04:20:00,618,22.000000,2179-03-06 04:00:00
6,100132,2179-03-06 04:25:00,211,82.000000,2179-03-06 04:00:00
7,100132,2179-03-06 02:40:00,455,82.000000,2179-03-06 03:00:00
8,100132,2179-03-06 02:30:00,8441,45.000000,2179-03-06 02:00:00
9,100132,2179-03-06 00:00:00,211,73.000000,2179-03-06 00:00:00


## SQL 쿼리 끝

## 환자 1명당 24 row를 가진 schema를 만들고, chart event merge
### patients df(hemo 포함), chart df는 csv로 저장되어있음 (csv load 후 실행)

In [28]:
event_df = pd.read_csv('/content/drive/MyDrive/의료인공지능/mimic_test/df_event.csv')
non_event_df = pd.read_csv('/content/drive/MyDrive/의료인공지능/mimic_test/df_non_event.csv')

event_chartdf = pd.read_csv('/content/drive/MyDrive/의료인공지능/mimic_test/chartdf_event.csv')
non_event_chartdf = pd.read_csv('/content/drive/MyDrive/의료인공지능/mimic_test/chartdf_non_event.csv')

In [80]:
## non-event // event 
df = event_df
chart_df = event_chartdf

In [81]:
chart_df

,Unnamed: 0,hadm_id,charttime,itemid,valuenum,charttime_ind
0,0,100132,2179-03-06 04:10:00,8441,43.0,2179-03-06 04:00:00
1,1,100132,2179-03-06 02:00:00,455,100.0,2179-03-06 02:00:00
2,2,100132,2179-03-06 02:35:00,455,78.0,2179-03-06 03:00:00
3,3,100132,2179-03-06 04:20:00,455,58.0,2179-03-06 04:00:00
4,4,100132,2179-03-06 02:25:00,8441,45.0,2179-03-06 02:00:00
...,...,...,...,...,...,...
60337,122,199780,2144-07-06 00:00:00,455,117.0,2144-07-06 00:00:00
60338,123,199780,2144-07-05 02:00:00,211,58.0,2144-07-05 02:00:00
60339,124,199780,2144-07-05 10:00:00,211,75.0,2144-07-05 10:00:00
60340,125,199780,2144-07-05 09:00:00,8441,72.0,2144-07-05 09:00:00


In [82]:
## drop outlier

sbp = [220050, 220179, 225309, 51, 455]
dbp = [220051, 220180, 225310, 8368, 8441]
resp = [220210, 618]
hr = [220045, 211] 
temp_c = [223762, 676, 677] 
spo2 = [646]

chart_df['valuenum'] = np.select(
    [
      chart_df['itemid'].isin(sbp) & (chart_df.valuenum > 0) & (chart_df.valuenum <= 300), 
      chart_df['itemid'].isin(dbp) & (chart_df.valuenum > 0) & (chart_df.valuenum <= 200),
      chart_df['itemid'].isin(resp) & (chart_df.valuenum > 11) & (chart_df.valuenum <= 80),
      chart_df['itemid'].isin(hr) & (chart_df.valuenum > 7) & (chart_df.valuenum <= 250),
      chart_df['itemid'].isin(temp_c) & (chart_df.valuenum > 12) & (chart_df.valuenum <= 150),
      chart_df['itemid'].isin(spo2) & (chart_df.valuenum > 13) & (chart_df.valuenum <= 90)
    ], 
    [
      chart_df['valuenum'],
      chart_df['valuenum'],
      chart_df['valuenum'],
      chart_df['valuenum'],
      chart_df['valuenum'],
      chart_df['valuenum']
    ], 
    default=np.nan
)

In [83]:
def get_data_per_id(df,id):

  per_id_hour_lab= pd.DataFrame()
  df_trial= df[df['hadm_id']==id][["charttime","itemid",'valuenum']].drop_duplicates(keep = 'last') 
  chart_trial = df_trial.pivot(index='charttime', columns='itemid', values='valuenum')
  for column_name in chart_trial.columns:
    per_id_hour_lab= per_id_hour_lab.append(pd.Series(chart_trial.loc[:,column_name],name=column_name).resample('h').mean()) ## 이상치 제거 필요
  
  data_per_id = per_id_hour_lab.T
  data_per_id['hadm_id'] = id ## hadm_id 추가

  return data_per_id

In [84]:
def check_number_col(df):

  sbp = [220050, 220179, 225309, 51, 455]
  dbp = [220051, 220180, 225310, 8368, 8441]
  resp = [220210, 618]
  hr = [220045, 211] 
  temp_c = [223762, 676, 677] 
  spo2 = [646] 

  num_col = 0
  lab_params = [sbp,dbp,resp,hr, temp_c, spo2]
  for params in lab_params:
    if (any(i in params for i in list(df.columns))):
      num_col+=1
  return num_col

In [85]:
def get_number_checked_data_per_id(total_df):

  id_list = []
  for id in sorted(list(set(total_df['hadm_id']))):
    if check_number_col(get_data_per_id(total_df, id)) >= 6:
      id_list.append(id)
  
  return id_list

In [86]:
def combine_same_item(output):

  sbp = [220050, 220179, 225309, 51, 455]
  dbp = [220051, 220180, 225310, 8368, 8441]
  resp = [220210, 618]
  hr = [220045, 211] 
  temp_c = [223762, 676, 677] 
  spo2 = [646] 

  col_names = ['sbp','dbp','resp','hr', 'temp_c', 'spo2']
  lab_params = [sbp,dbp,resp,hr, temp_c, spo2]
  lab_dict = dict(zip(col_names, lab_params))

  for col, item_id in lab_dict.items():
    for i in range(len(item_id)):
      if item_id[i] in output.columns:
        output[col] = np.nan
        output[col] = np.where(output[col].isnull(), output[item_id[i]], output[col])

  var_list = ['hadm_id', 455, 8441, 618, 211, 677, 646]
  output = output[var_list]
  output = output.rename(columns = {455:'sbp',8441:'dbp',618:'resp',211:'hr',677:'temp_c',646:'spo2'})
                
  return output

In [87]:
chart_df['charttime'] = pd.to_datetime(chart_df['charttime'])
chart_df['charttime_ind'] = pd.to_datetime(chart_df['charttime_ind'])

In [88]:
## 각 vital sign들의 item_id 중 하나 이상 가지고있는 환자 id 추출
id_list = get_number_checked_data_per_id(chart_df)

In [89]:
output = pd.DataFrame()

for id in id_list:
  tmp_output = get_data_per_id(chart_df,id)
  output = tmp_output.append(output)

In [90]:
output

,211,455,618,646,677,678,8441,hadm_id,51,8368,676,679
2144-07-05 01:00:00,57.000000,90.000000,18.0,NaN,36.666698,NaN,34.000000,199780,NaN,NaN,NaN,NaN
2144-07-05 02:00:00,58.000000,84.000000,NaN,NaN,NaN,NaN,47.000000,199780,NaN,NaN,NaN,NaN
2144-07-05 03:00:00,58.000000,90.000000,NaN,NaN,36.777802,NaN,52.000000,199780,NaN,NaN,NaN,NaN
2144-07-05 04:00:00,63.000000,93.000000,20.0,NaN,NaN,NaN,39.000000,199780,NaN,NaN,NaN,NaN
2144-07-05 05:00:00,56.000000,103.000000,20.0,NaN,NaN,NaN,40.000000,199780,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2179-03-06 00:00:00,73.000000,98.000000,15.0,NaN,NaN,NaN,55.000000,100132,NaN,NaN,NaN,NaN
2179-03-06 01:00:00,85.000000,105.000000,20.0,90.0,NaN,NaN,55.500000,100132,NaN,NaN,NaN,NaN
2179-03-06 02:00:00,83.666667,88.833333,15.5,NaN,NaN,NaN,44.833333,100132,NaN,NaN,NaN,NaN
2179-03-06 03:00:00,82.000000,91.000000,15.0,NaN,NaN,NaN,44.000000,100132,NaN,NaN,NaN,NaN


In [91]:
result = combine_same_item(output)
var_list = ['hadm_id', 'sbp', 'dbp', 'resp', 'hr', 'temp_c', 'spo2']
result = result.loc[:,var_list]

In [92]:
result

,hadm_id,sbp,dbp,resp,hr,temp_c,spo2
2144-07-05 01:00:00,199780,90.000000,34.000000,18.0,57.000000,36.666698,NaN
2144-07-05 02:00:00,199780,84.000000,47.000000,NaN,58.000000,NaN,NaN
2144-07-05 03:00:00,199780,90.000000,52.000000,NaN,58.000000,36.777802,NaN
2144-07-05 04:00:00,199780,93.000000,39.000000,20.0,63.000000,NaN,NaN
2144-07-05 05:00:00,199780,103.000000,40.000000,20.0,56.000000,NaN,NaN
...,...,...,...,...,...,...,...
2179-03-06 00:00:00,100132,98.000000,55.000000,15.0,73.000000,NaN,NaN
2179-03-06 01:00:00,100132,105.000000,55.500000,20.0,85.000000,NaN,90.0
2179-03-06 02:00:00,100132,88.833333,44.833333,15.5,83.666667,NaN,NaN
2179-03-06 03:00:00,100132,91.000000,44.000000,15.0,82.000000,NaN,NaN


In [93]:
df['charttime'] = pd.to_datetime(df['charttime'])

In [94]:
## 1 case 당 24행 df를 만들어 붙여넣는 작업

df_schema = pd.DataFrame()

for i in range(len(df)):
  tmp_output = pd.DataFrame({'hadm_id': df['hadm_id'][i], 
                     'time': pd.date_range(df['charttime'][i]-timedelta(1), df['charttime'][i] , freq="60min")}
                    )
  tmp_output = tmp_output.iloc[1:]
  df_schema = tmp_output.append(df_schema)

In [95]:
df_schema['time'] = df_schema['time'].dt.floor('H')
df_schema = df_schema.drop_duplicates(['hadm_id', 'time'], keep='first')
df_schema = df_schema.set_index(['hadm_id', 'time'])
df_schema

Empty DataFrame
Columns: []
Index: [(199780, 2144-07-05 02:00:00), (199780, 2144-07-05 03:00:00), (199780, 2144-07-05 04:00:00), (199780, 2144-07-05 05:00:00), (199780, 2144-07-05 06:00:00), (199780, 2144-07-05 07:00:00), (199780, 2144-07-05 08:00:00), (199780, 2144-07-05 09:00:00), (199780, 2144-07-05 10:00:00), (199780, 2144-07-05 11:00:00), (199780, 2144-07-05 12:00:00), (199780, 2144-07-05 13:00:00), (199780, 2144-07-05 14:00:00), (199780, 2144-07-05 15:00:00), (199780, 2144-07-05 16:00:00), (199780, 2144-07-05 17:00:00), (199780, 2144-07-05 18:00:00), (199780, 2144-07-05 19:00:00), (199780, 2144-07-05 20:00:00), (199780, 2144-07-05 21:00:00), (199780, 2144-07-05 22:00:00), (199780, 2144-07-05 23:00:00), (199780, 2144-07-06 00:00:00), (199780, 2144-07-06 01:00:00), (199589, 2146-10-01 16:00:00), (199589, 2146-10-01 17:00:00), (199589, 2146-10-01 18:00:00), (199589, 2146-10-01 19:00:00), (199589, 2146-10-01 20:00:00), (199589, 2146-10-01 21:00:00), (199589, 2146-10-01 22:00:00), (199589, 2146-10-01 23:00:00), (199589, 2146-10-02 00:00:00), (199589, 2146-10-02 01:00:00), (199589, 2146-10-02 02:00:00), (199589, 2146-10-02 03:00:00), (199589, 2146-10-02 04:00:00), (199589, 2146-10-02 05:00:00), (199589, 2146-10-02 06:00:00), (199589, 2146-10-02 07:00:00), (199589, 2146-10-02 08:00:00), (199589, 2146-10-02 09:00:00), (199589, 2146-10-02 10:00:00), (199589, 2146-10-02 11:00:00), (199589, 2146-10-02 12:00:00), (199589, 2146-10-02 13:00:00), (199589, 2146-10-02 14:00:00), (199589, 2146-10-02 15:00:00), (199365, 2111-01-20 05:00:00), (199365, 2111-01-20 06:00:00), (199365, 2111-01-20 07:00:00), (199365, 2111-01-20 08:00:00), (199365, 2111-01-20 09:00:00), (199365, 2111-01-20 10:00:00), (199365, 2111-01-20 11:00:00), (199365, 2111-01-20 12:00:00), (199365, 2111-01-20 13:00:00), (199365, 2111-01-20 14:00:00), (199365, 2111-01-20 15:00:00), (199365, 2111-01-20 16:00:00), (199365, 2111-01-20 17:00:00), (199365, 2111-01-20 18:00:00), (199365, 2111-01-20 19:00:00), (199365, 2111-01-20 20:00:00), (199365, 2111-01-20 21:00:00), (199365, 2111-01-20 22:00:00), (199365, 2111-01-20 23:00:00), (199365, 2111-01-21 00:00:00), (199365, 2111-01-21 01:00:00), (199365, 2111-01-21 02:00:00), (199365, 2111-01-21 03:00:00), (199365, 2111-01-21 04:00:00), (199151, 2129-08-28 01:00:00), (199151, 2129-08-28 02:00:00), (199151, 2129-08-28 03:00:00), (199151, 2129-08-28 04:00:00), (199151, 2129-08-28 05:00:00), (199151, 2129-08-28 06:00:00), (199151, 2129-08-28 07:00:00), (199151, 2129-08-28 08:00:00), (199151, 2129-08-28 09:00:00), (199151, 2129-08-28 10:00:00), (199151, 2129-08-28 11:00:00), (199151, 2129-08-28 12:00:00), (199151, 2129-08-28 13:00:00), (199151, 2129-08-28 14:00:00), (199151, 2129-08-28 15:00:00), (199151, 2129-08-28 16:00:00), (199151, 2129-08-28 17:00:00), (199151, 2129-08-28 18:00:00), (199151, 2129-08-28 19:00:00), (199151, 2129-08-28 20:00:00), (199151, 2129-08-28 21:00:00), (199151, 2129-08-28 22:00:00), (199151, 2129-08-28 23:00:00), (199151, 2129-08-29 00:00:00), (198630, 2125-02-09 00:00:00), (198630, 2125-02-09 01:00:00), (198630, 2125-02-09 02:00:00), (198630, 2125-02-09 03:00:00), ...]

[10200 rows x 0 columns]

In [96]:
result.rename_axis('time', inplace=True)
result.reset_index(inplace=True)
result = result.set_index(['hadm_id', 'time'])
result

sbp        dbp  ...     temp_c  spo2
hadm_id time                                        ...                 
199780  2144-07-05 01:00:00   90.000000  34.000000  ...  36.666698   NaN
        2144-07-05 02:00:00   84.000000  47.000000  ...        NaN   NaN
        2144-07-05 03:00:00   90.000000  52.000000  ...  36.777802   NaN
        2144-07-05 04:00:00   93.000000  39.000000  ...        NaN   NaN
        2144-07-05 05:00:00  103.000000  40.000000  ...        NaN   NaN
...                                 ...        ...  ...        ...   ...
100132  2179-03-06 00:00:00   98.000000  55.000000  ...        NaN   NaN
        2179-03-06 01:00:00  105.000000  55.500000  ...        NaN  90.0
        2179-03-06 02:00:00   88.833333  44.833333  ...        NaN   NaN
        2179-03-06 03:00:00   91.000000  44.000000  ...        NaN   NaN
        2179-03-06 04:00:00   87.888889  43.222222  ...  36.444401  89.0

[5494 rows x 6 columns]

In [97]:
result_combined = df_schema.merge(result, left_index=False, right_index=True, left_on= ['hadm_id', 'time'] ,how = 'left')
result_combined['event'] = 1

In [98]:
## carry-forward
result_combined = result_combined.groupby('hadm_id').apply(lambda x:x.fillna(method='ffill').fillna(method='bfill'))
result_combined = result_combined.dropna() ## 24시간 이내 수치가 하나도 없는 column이 하나라도 있으면 drop

In [99]:
result_combined

sbp         dbp  ...  spo2  event
hadm_id time                                         ...             
199365  2111-01-20 05:00:00  145.000000  101.000000  ...  90.0      1
        2111-01-20 06:00:00  101.000000   70.000000  ...  90.0      1
        2111-01-20 07:00:00  127.000000   73.000000  ...  90.0      1
        2111-01-20 08:00:00  112.300000   73.700000  ...  90.0      1
        2111-01-20 09:00:00  127.000000   79.000000  ...  90.0      1
...                                 ...         ...  ...   ...    ...
100132  2179-03-06 00:00:00   98.000000   55.000000  ...  90.0      1
        2179-03-06 01:00:00  105.000000   55.500000  ...  90.0      1
        2179-03-06 02:00:00   88.833333   44.833333  ...  90.0      1
        2179-03-06 03:00:00   91.000000   44.000000  ...  90.0      1
        2179-03-06 04:00:00   87.888889   43.222222  ...  89.0      1

[1128 rows x 7 columns]

## BOLUS DATA 만들기

In [51]:
crystalloid_list1 = [225158, 
    225828, 
    225944, 
    225797, 
	  225159, 
	  225823, 
	  225825, 
	  225827, 
	  225941, 
	  226089]

crystalloid_list2 = [
    30015 
  , 30018 
  , 30020 
  , 30021 
  , 30058 
  , 30060 
  , 30061 
  , 30063 
  , 30065 
  , 30159 
  , 30160 
  , 30169 
  , 30190 
  , 40850 
  , 41491 
  , 42639 
  , 42187 
  , 43819 
  , 41430 
  , 40712 
  , 44160 
  , 42383 
  , 42297 
  , 42453 
  , 40872 
  , 41915 
  , 41490 
  , 46501 
  , 45045 
  , 41984 
  , 41371 
  , 41582 
  , 41322 
  , 40778 
  , 41896 
  , 41428 
  , 43936 
  , 44200 
  , 41619 
  , 40424 
  , 41457 
  , 41581 
  , 42844 
  , 42429 
  , 41356 
  , 40532 
  , 42548 
  , 44184 
  , 44521 
  , 44741 
  , 44126 
  , 44110 
  , 44633 
  , 44983 
  , 44815 
  , 43986 
  , 45079 
  , 46781 
  , 45155 
  , 43909 
  , 41467 
  , 44367 
  , 41743 
  , 40423 
  , 44263 
  , 42749 
  , 45480 
  , 44491 
  , 41695 
  , 46169 
  , 41580 
  , 41392 
  , 45989 
  , 45137 
  , 45154 
  , 44053 
  , 41416 
  , 44761 
  , 41237 
  , 44426 
  , 43975 
  , 44894 
  , 41380 
  , 42671] 

In [52]:
crystalloid_df = pd.DataFrame()
for id in id_list:
    SQL = """with t1 as
          (
            select
              mv.hadm_id
            , mv.starttime as charttime
            -- standardize the units to millilitres
            -- also metavision has floating point precision.. but we only care down to the mL
            , round(case
                when mv.amountuom = 'L'
                  then mv.amount * 1000.0
                when mv.amountuom = 'ml'
                  then mv.amount
              else null end) as amount
            from physionet-data.mimiciii_clinical.inputevents_mv as mv
            where mv.itemid in {x}
            and mv.statusdescription != 'Rewritten'
            and
            -- in MetaVision, these ITEMIDs appear with a null rate IFF endtime=starttime + 1 minute
            -- so it is sufficient to:
            --    (1) check the rate is > 240 if it exists or
            --    (2) ensure the rate is null and amount > 240 ml
              (
                (mv.rate is not null and mv.rateuom = 'mL/hour' and mv.rate > 248)
                OR (mv.rate is not null and mv.rateuom = 'mL/min' and mv.rate > (248/60.0))
                OR (mv.rate is null and mv.amountuom = 'L' and mv.amount > 0.248)
                OR (mv.rate is null and mv.amountuom = 'ml' and mv.amount > 248)
              )
          )
          , t2 as
          (
            select
              cv.hadm_id
            , cv.charttime
            -- carevue always has units in millilitres
            , round(cv.amount) as amount
            from physionet-data.mimiciii_clinical.inputevents_cv as cv
            where cv.itemid in {y}
            and cv.amount > 248
            and cv.amount <= 2000
            and cv.amountuom = 'ml'
            )
          select 
              hadm_id
            , charttime
            , sum(amount) as crystalloid_bolus
          from t1
          -- just because the rate was high enough, does *not* mean the final amount was
          where amount > 248
          group by t1.hadm_id, t1.charttime
          UNION ALL
          select
              hadm_id
            , charttime
            , sum(amount) as crystalloid_bolus
          from t2
          WHERE hadm_id = {z}
          group by t2.hadm_id, t2.charttime
          order by hadm_id, charttime
          ;""".format(x=tuple(crystalloid_list1), y=tuple(crystalloid_list2), z=id)
    adddf = pd.io.gbq.read_gbq(SQL, project_id = project_id, dialect = 'standard')
    crystalloid_df=crystalloid_df.append(adddf)

In [53]:
colloid_list1 = [220864,
220862, 
225174, 
225795, 
225796  
]

colloid_list2 = [30008
,30009 
,42832 
,40548 
,45403 
,44203 
,30181 
,46564 
,43237 
,43353 
,30012 
,46313 
,30011 
,30016 
,42975 
,42944 
,46336 
,46729 
,40033 
,45410 
,42731 ]

colloid_list3 = [2510 
, 3087 
, 6937 
, 3087 
, 3088]

In [54]:
colloid_df = pd.DataFrame()
for id in id_list:
    SQL = """with t1 as
          (
            select
              mv.hadm_id
            , mv.starttime as charttime
            -- standardize the units to millilitres
            -- also metavision has floating point precision.. but we only care down to the mL
            , round(case
                when mv.amountuom = 'L'
                  then mv.amount * 1000.0
                when mv.amountuom = 'ml'
                  then mv.amount
              else null end) as amount
            from physionet-data.mimiciii_clinical.inputevents_mv as mv
            where mv.itemid in {x}
            and mv.statusdescription != 'Rewritten'
            and
            -- in MetaVision, these ITEMIDs never appear with a null rate
            -- so it is sufficient to check the rate is > 100
              (
                (mv.rateuom = 'mL/hour' and mv.rate > 100)
                OR (mv.rateuom = 'mL/min' and mv.rate > (100/60.0))
                OR (mv.rateuom = 'mL/kg/hour' and (mv.rate*mv.patientweight) > 100)
              )
          )
          , t2 as
          (
            select
              cv.hadm_id
            , cv.charttime
            -- carevue always has units in millilitres (or null)
            , round(cv.amount) as amount
            from physionet-data.mimiciii_clinical.inputevents_cv as cv
            where cv.itemid in {y}
            and cv.amount > 100
            and cv.amount < 2000
          )
          -- some colloids are charted in chartevents
          , t3 as
          (
            select
              ce.hadm_id
            , ce.charttime
            -- carevue always has units in millilitres (or null)
            , round(ce.valuenum) as amount
            from physionet-data.mimiciii_clinical.chartevents as ce
            where ce.itemid in {z}
            and ce.valuenum is not null
            and ce.valuenum > 100
            and ce.valuenum < 2000
          )
          select
              hadm_id
            , charttime
            , sum(amount) as colloid_bolus
          from t1
          -- just because the rate was high enough, does *not* mean the final amount was
          where amount > 100
          group by t1.hadm_id, t1.charttime
          UNION ALL
          select
              hadm_id
            , charttime
            , sum(amount) as colloid_bolus
          from t2
          group by t2.hadm_id, t2.charttime
          UNION ALL 
          select
              hadm_id
            , charttime
            , sum(amount) as colloid_bolus
          from t3
          WHERE hadm_id = {w}
          group by t3.hadm_id, t3.charttime
          order by hadm_id, charttime;
          """.format(x=tuple(colloid_list1), y=tuple(colloid_list2), z=tuple(colloid_list3), w=id)
    adddf = pd.io.gbq.read_gbq(SQL, project_id = project_id, dialect = 'standard')
    colloid_df=colloid_df.append(adddf)

In [55]:
rbc_list1 = [    30179,  
    30001, 
    30004,
    42324,  
    42588,  
    42239,  
    46407,  
    46612,  
    46124,  
    42740 
]

rbc_list2 = [225168, 227070]

In [56]:
rbc_df = pd.DataFrame()
for id in id_list:
    SQL = """
WITH rbc_table AS(
  SELECT cv.hadm_id, cv.charttime,
        CASE WHEN cv.itemid in {x} OR mv.itemid in {y}
THEN 1
ELSE 0
END AS rbc
FROM physionet-data.mimiciii_clinical.inputevents_cv as cv
LEFT JOIN physionet-data.mimiciii_clinical.inputevents_mv AS mv
  ON cv.hadm_id = mv.hadm_id)
SELECT hadm_id, charttime, rbc
FROM rbc_table
WHERE hadm_id = {z}
ORDER BY hadm_id


;""".format(x=tuple(rbc_list1), y=tuple(rbc_list2), z=id)
    adddf = pd.io.gbq.read_gbq(SQL, project_id = project_id, dialect = 'standard')
    rbc_df=rbc_df.append(adddf)

In [57]:
urine_list = [40055,
  43175, 
  40069, 
  40094, 
  40715, 
  40473, 
  40085, 
  40057, 
  40056, 
  40405, 
  40428, 
  40086, 
  40096, 
  40651, 
  226559,
  226560,
  226561,
  226584,
  226563,
  226564,
  226565,
  226567,
  226557,
  226558,
  227488,
  227489 
]

In [58]:
urine_df = pd.DataFrame()
for id in id_list:
    SQL = """
  select oe.hadm_id, oe.charttime
  , SUM(
      -- we consider input of GU irrigant as a negative volume
      case when oe.itemid = 227488 then -1*value
      else value end
    ) as urine
  from physionet-data.mimiciii_clinical.outputevents as oe
  where oe.itemid in {x}
  and hadm_id = {y}
  and oe.value < 5000 -- sanity check on urine value
  and oe.hadm_id is not null
  group by hadm_id, charttime
;""".format(x=tuple(urine_list), y=id)
    adddf = pd.io.gbq.read_gbq(SQL, project_id = project_id, dialect = 'standard')
    urine_df=urine_df.append(adddf)

In [60]:
with open("/content/drive/MyDrive/의료인공지능/mimic_test/crystalloid_df_non_event.pickle","wb") as fw:
    pickle.dump(crystalloid_df, fw)

with open("/content/drive/MyDrive/의료인공지능/mimic_test/colloid_df_non_event.pickle","wb") as fw:
    pickle.dump(colloid_df, fw)

with open("/content/drive/MyDrive/의료인공지능/mimic_test/rbc_df_non_event.pickle","wb") as fw:
    pickle.dump(rbc_df, fw)

with open("/content/drive/MyDrive/의료인공지능/mimic_test/urine_df_non_event.pickle","wb") as fw:
    pickle.dump(urine_df, fw)

## load pickles

In [100]:
 with open('/content/drive/MyDrive/의료인공지능/mimic_test/crystalloid_df_event.pickle', 'rb') as f:
     crystalloid_df = pickle.load(f)

 with open('/content/drive/MyDrive/의료인공지능/mimic_test/colloid_df_event.pickle', 'rb') as f:
     colloid_df = pickle.load(f)

 with open('/content/drive/MyDrive/의료인공지능/mimic_test/rbc_df_event.pickle', 'rb') as f:
     rbc_df = pickle.load(f)

 with open('/content/drive/MyDrive/의료인공지능/mimic_test/urine_df_event.pickle', 'rb') as f:
     urine_df = pickle.load(f)

In [101]:
def matching_schema(df):

  df = df.dropna()
  df = df.drop_duplicates()
  df = df.assign(charttime_ind=df.charttime.dt.round('H'))

  df['charttime_ind'] = df['charttime'].dt.round('H')
  df.rename(columns={'charttime_ind': 'time'}, inplace = True)
  df = df.drop('charttime',axis=1)
  df = df.set_index(['hadm_id','time'])

  return df

In [102]:
result_combined_r = result_combined.reset_index(drop=False, col_fill='index')

In [103]:
df_list = [crystalloid_df, colloid_df, rbc_df, urine_df]

for i in range(len(df_list)):
  df = df_list[i]
  df = matching_schema(df)
  result_combined_r = result_combined_r.merge( df,  how='left', left_on=['hadm_id','time'], right_on = ['hadm_id','time'])

In [104]:
result_combined_r = result_combined_r.drop_duplicates(['hadm_id','time'])

In [105]:
result_combined_r

,hadm_id,time,sbp,dbp,resp,hr,temp_c,spo2,event,crystalloid_bolus,colloid_bolus,rbc,urine
0,199365,2111-01-20 05:00:00,145.000000,101.000000,15.000000,107.000000,35.555599,90.0,1,NaN,NaN,0.0,NaN
1,199365,2111-01-20 06:00:00,101.000000,70.000000,12.666667,93.333333,35.555599,90.0,1,NaN,NaN,0.0,160.0
5,199365,2111-01-20 07:00:00,127.000000,73.000000,15.000000,100.000000,35.555599,90.0,1,NaN,NaN,0.0,NaN
6,199365,2111-01-20 08:00:00,112.300000,73.700000,13.166667,89.500000,35.555599,90.0,1,NaN,NaN,0.0,240.0
13,199365,2111-01-20 09:00:00,127.000000,79.000000,13.166667,93.166667,35.555599,90.0,1,NaN,NaN,0.0,80.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363,100132,2179-03-06 00:00:00,98.000000,55.000000,15.000000,73.000000,36.166698,90.0,1,NaN,NaN,NaN,NaN
1364,100132,2179-03-06 01:00:00,105.000000,55.500000,20.000000,85.000000,36.166698,90.0,1,NaN,NaN,0.0,NaN
1365,100132,2179-03-06 02:00:00,88.833333,44.833333,15.500000,83.666667,36.166698,90.0,1,NaN,NaN,NaN,NaN
1366,100132,2179-03-06 03:00:00,91.000000,44.000000,15.000000,82.000000,36.166698,90.0,1,500.0,NaN,0.0,NaN


In [106]:
## binary
result_combined_r['crystalloid_bolus_i'] = np.where(result_combined_r['crystalloid_bolus'].notnull(), 1,0)
result_combined_r['colloid_bolus_i'] = np.where(result_combined_r['colloid_bolus'].notnull(), 1,0)
result_combined_r['urine_i'] = np.where(result_combined_r['urine'].notnull(), 1,0)

result_combined_r['crystalloid_bolus'] = np.where(result_combined_r['crystalloid_bolus'].notnull(), result_combined_r['crystalloid_bolus'],0)
result_combined_r['colloid_bolus'] = np.where(result_combined_r['colloid_bolus'].notnull(), result_combined_r['colloid_bolus'],0)
result_combined_r['rbc'] = np.where(result_combined_r['rbc'].notnull(), result_combined_r['rbc'],0)
result_combined_r['urine'] = np.where(result_combined_r['urine'].notnull(), result_combined_r['urine'],0)

result_combined_r

,hadm_id,time,sbp,dbp,resp,hr,temp_c,spo2,event,crystalloid_bolus,colloid_bolus,rbc,urine,crystalloid_bolus_i,colloid_bolus_i,urine_i
0,199365,2111-01-20 05:00:00,145.000000,101.000000,15.000000,107.000000,35.555599,90.0,1,0.0,0.0,0.0,0.0,0,0,0
1,199365,2111-01-20 06:00:00,101.000000,70.000000,12.666667,93.333333,35.555599,90.0,1,0.0,0.0,0.0,160.0,0,0,1
5,199365,2111-01-20 07:00:00,127.000000,73.000000,15.000000,100.000000,35.555599,90.0,1,0.0,0.0,0.0,0.0,0,0,0
6,199365,2111-01-20 08:00:00,112.300000,73.700000,13.166667,89.500000,35.555599,90.0,1,0.0,0.0,0.0,240.0,0,0,1
13,199365,2111-01-20 09:00:00,127.000000,79.000000,13.166667,93.166667,35.555599,90.0,1,0.0,0.0,0.0,80.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363,100132,2179-03-06 00:00:00,98.000000,55.000000,15.000000,73.000000,36.166698,90.0,1,0.0,0.0,0.0,0.0,0,0,0
1364,100132,2179-03-06 01:00:00,105.000000,55.500000,20.000000,85.000000,36.166698,90.0,1,0.0,0.0,0.0,0.0,0,0,0
1365,100132,2179-03-06 02:00:00,88.833333,44.833333,15.500000,83.666667,36.166698,90.0,1,0.0,0.0,0.0,0.0,0,0,0
1366,100132,2179-03-06 03:00:00,91.000000,44.000000,15.000000,82.000000,36.166698,90.0,1,500.0,0.0,0.0,0.0,1,0,0


In [107]:
result_combined_r.to_csv("/content/drive/MyDrive/의료인공지능/mimic_test/time_series_event.csv")

In [109]:
df1 = pd.read_csv('/content/drive/MyDrive/의료인공지능/mimic_test/time_series_event.csv')
df2 = pd.read_csv('/content/drive/MyDrive/의료인공지능/mimic_test/time_series_non_event.csv')


result = pd.concat([df1,df2])
result.to_csv("/content/drive/MyDrive/의료인공지능/mimic_test/time_series_total.csv")